## Combine my data with stuff from Kaggle

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_mine = pd.read_csv('./covid19.csv')
df_mine.shape

(101, 8)

In [3]:
df_kag = pd.read_csv('./kaggle/covid19countryinfo.csv')
df_kag.columns = ['Country', *df_kag.columns[1:]]
df_kag = df_kag[['Country', 'tests', 'testpop', 'density', 'urbanpop', 'quarantine', 'schools', 'restrictions', 'hospibed']]
df_kag.replace({'Korea, South': 'South Korea', 'Czechia': 'Czech Republic', 'Taiwan*': 'Taiwan', 'US': 'United States'}, inplace=True)

In [4]:
df = df_kag.merge(df_mine, how="inner", on="Country")
df.head()

,Country,tests,testpop,density,urbanpop,quarantine,schools,restrictions,hospibed,Total Infected,GDP 2018,Crime Index,Population 2020,Smoking 2016,Females 2018,Median Age
0,Albania,NaN,NaN,105.0,63.0,NaN,NaN,NaN,2.9,123,1.510250e+10,40.02,2877.797,28.7,49.063095,32.9
1,Algeria,NaN,NaN,18.0,73.0,NaN,NaN,NaN,1.9,264,1.737580e+11,54.41,43851.044,15.6,49.484268,28.1
2,Argentina,NaN,NaN,17.0,93.0,3/20/2020,NaN,NaN,5.0,387,5.198715e+11,62.96,45195.774,21.8,51.237348,31.7
3,Armenia,694.0,4269.802594,104.0,63.0,NaN,NaN,NaN,4.2,249,1.243309e+10,20.78,2963.243,24.1,52.956577,35.1
4,Australia,31635.0,806.065560,3.0,86.0,NaN,NaN,3/23/2020,3.8,2044,1.433904e+12,42.70,25499.884,14.7,50.199623,38.7


In [5]:
df.columns = ['Country', 'Tests', 'Test Pop', 'Density', 'Urban Pop', 'Quarantine', 'Schools', 'Restrictions', 'Hospital Bed', *df_mine.columns[1:]]
df.shape

(97, 16)

In [6]:
# data cleaning and summing total events by country
def preprocess_summation(df, description):
    df = df.drop(['Province/State', 'Lat', 'Long'], axis=1).groupby(['Country/Region']).sum().reset_index()
    col_name = 'Total' + ' ' + description
    df[col_name] = df.apply(lambda r: r[-1], axis=1)
    df = df[['Country/Region', col_name]]
    df.columns = ['Country', col_name]
    df.replace({'Korea, South': 'South Korea', 'Czechia': 'Czech Republic', 'Taiwan*': 'Taiwan', 'US': 'United States'}, inplace=True)
    return df

In [7]:
df_recover = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-deaths.csv')
total_recover = preprocess_summation(df_recover, 'Deaths')
df = df.merge(total_recover, how='inner', on='Country')

In [8]:
cols = df.columns.tolist()
cols[len(cols)-2], cols[9] = cols[9], cols[len(cols)-2]
df = df[cols]
df.head()
df[df.Country == 'India']

,Country,Tests,Test Pop,Density,Urban Pop,Quarantine,Schools,Restrictions,Hospital Bed,Median Age,GDP 2018,Crime Index,Population 2020,Smoking 2016,Females 2018,Total Infected,Total Deaths
39,India,5900.0,233899.0483,464.0,35.0,3/23/2020,NaN,NaN,0.7,27.9,2.718732e+12,42.38,1380004.385,11.5,48.02354,536,7.0


In [9]:
df.to_csv('covid19_by_country.csv', index=False)